In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Battery_RUL.csv')

In [3]:
df.dtypes

Cycle_Index                  float64
Discharge Time (s)           float64
Decrement 3.6-3.4V (s)       float64
Max. Voltage Dischar. (V)    float64
Min. Voltage Charg. (V)      float64
Time at 4.15V (s)            float64
Time constant current (s)    float64
Charging time (s)            float64
RUL                            int64
dtype: object

In [4]:
df.drop(['Cycle_Index', 'Discharge Time (s)', 'Decrement 3.6-3.4V (s)', 'Time constant current (s)', 'Charging time (s)'], axis=1, inplace=True)

In [5]:
X = df.drop(['RUL'], axis=1)
y = df['RUL']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [6]:
class Pipeline:
  def __init__(self, scaler):
    self.scaler = scaler
  def fit(self, X, y):
    X = self.scaler.fit_transform(X, y)
    return X, y
  def transform(self, X, y):
    X = self.scaler.transform(X)
    return X, y

In [7]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
pipeline = Pipeline(scaler)
X_train, y_train = pipeline.fit(X_train, y_train)
X_test, y_test = pipeline.transform(X_test, y_test)

# Self Neuralizing Neural Network

In [11]:
from tensorflow.keras import backend as K

def rmse(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [12]:
from tensorflow.keras.initializers import lecun_normal
from tensorflow.keras.regularizers import l2

model = keras.Sequential([
    keras.layers.Dense(16, input_shape=(3,), activation='selu', kernel_initializer=lecun_normal(), kernel_regularizer=l2(0.01)),
    keras.layers.Dense(8, activation='selu', kernel_initializer=lecun_normal(), kernel_regularizer=l2(0.01)),
    keras.layers.Dense(1)
])
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=[rmse])
model.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
264/264 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 412704.6250 - rmse: 640.8116 - val_loss: 395087.4688 - val_rmse: 627.2278
Epoch 2/50
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 392598.7188 - rmse: 624.5948 - val_loss: 361929.6562 - val_rmse: 600.3038
Epoch 3/50
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 358318.3125 - rmse: 596.4568 - val_loss: 301036.6250 - val_rmse: 547.4417
Epoch 4/50
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 288688.1875 - rmse: 535.2922 - val_loss: 224814.4219 - val_rmse: 472.8263
Epoch 5/50
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 205084.2344 - rmse: 450.9012 - val_loss: 144739.2656 - val_rmse: 379.2164
Epoch 6/50
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 126339.0234 - rmse: 353.2867 - val_loss: 78050.7656 - val_rmse: 277.7756
Epoch 7/50
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 64082.6055 - rmse: 250.9113 - val_loss: 38925.2891 - val_rmse: 194.1157
Epoch 8/50
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 29348

In [13]:
_, rmse_train = model.evaluate(X_train, y_train)
_, rmse_test = model.evaluate(X_test, y_test)

print(f'Train RMSE: {rmse_train:.2f}')
print(f'Test RMSE: {rmse_test:.2f}')

330/330 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4736.8564 - rmse: 63.9008
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5182.0586 - rmse: 65.7998
Train RMSE: 66.02
Test RMSE: 66.17
